In [1]:
import sys
print(sys.executable)

/opt/anaconda3/bin/python


In [2]:
import pandas as pd
import seaborn as sns
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize

In [3]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [4]:
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from bs4 import BeautifulSoup

In [5]:
import os

In [6]:
df = pd.read_csv("imdb.csv")

In [7]:
df.head

<bound method NDFrame.head of                                                   review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]>

In [8]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2)

In [9]:
train.head

<bound method NDFrame.head of                                                   review sentiment
14801  I went to this movie only because I was dragge...  positive
28461  There is one great moment in *Surviving Christ...  negative
6815   Well, I saw that yesterday and It was much bet...  positive
47481  This was such a waste of time. Danger: If you ...  negative
40180  The Ascent (1977) <br /><br />Larisa Shepitko ...  positive
...                                                  ...       ...
5611   I have no idea what the producers of The Shiel...  negative
12331  Everyone has a first love, and though it is ha...  positive
46094  On a dark, gloomy New Year's Eve night, an ill...  positive
29644  As you can tell from the few comments posted h...  positive
35151  even though this movie is quite old, no matter...  positive

[40000 rows x 2 columns]>

preprocessing

In [10]:
ptrain = train[train.sentiment == "positive"]

In [11]:
ntrain = train[train.sentiment == "negative"]

In [12]:
ntrain.head

<bound method NDFrame.head of                                                   review sentiment
28461  There is one great moment in *Surviving Christ...  negative
47481  This was such a waste of time. Danger: If you ...  negative
46230  I rented this movie, knowing that it would be ...  negative
7169   Genie (Zoe Trilling) arrives in Egypt to visit...  negative
25640  Hilariously obvious "drama" about a bunch of h...  negative
...                                                  ...       ...
14906  Look carefully at the wonderful assortment of ...  negative
49797  Yeah i bought camp blood and it wasted about 8...  negative
30551  You want the worst horror movie of the 21st ce...  negative
18310  What a terrible sequel. The reason I give this...  negative
5611   I have no idea what the producers of The Shiel...  negative

[20051 rows x 2 columns]>

In [13]:
ptrain.head

<bound method NDFrame.head of                                                   review sentiment
14801  I went to this movie only because I was dragge...  positive
6815   Well, I saw that yesterday and It was much bet...  positive
40180  The Ascent (1977) <br /><br />Larisa Shepitko ...  positive
34416  This movie embodies the soul of modern "elite"...  positive
43695  This is film-making at it's simplest and it's ...  positive
...                                                  ...       ...
7164   Everyone knows about this ''Zero Day'' event. ...  positive
12331  Everyone has a first love, and though it is ha...  positive
46094  On a dark, gloomy New Year's Eve night, an ill...  positive
29644  As you can tell from the few comments posted h...  positive
35151  even though this movie is quite old, no matter...  positive

[19949 rows x 2 columns]>

In [14]:
negativereviews = []
for review in ntrain["review"]:
    review = review.replace("<br />", " ")
    doc = nlp(review)
    tokenbyreview = []
    ##print(review)
    for token in doc:
        if token.is_stop==False and token.is_punct==False and token.text.isnumeric()==False:
            tokenbyreview.append(token.text.lower())
            ##print(token.text.lower())
    ##print(tokenbyreview)
    negativereviews.append(set(tokenbyreview))

In [15]:
##negativereviews

In [16]:
count_negativetrain = {}
for bag in negativereviews:
    for word in bag:
        if word not in count_negativetrain:
            count_negativetrain[word]=1
        else: 
            count_negativetrain[word]+=1
##print(sorted(list(count_negativetrain.items()), key=lambda x: x[1], reverse=True)) 

In [17]:
positivereviews = []
for review in ptrain["review"]:
    review = review.replace("<br />", " ")
    doc = nlp(review)
    tokenbypreview = []
    ##print(review)
    for token in doc:
        if token.is_stop==False and token.is_punct==False and token.text.isnumeric()==False:
            tokenbypreview.append(token.text.lower())
            ##print(token.text.lower())
    ##print(tokenbyreview)
    positivereviews.append(set(tokenbypreview))

In [18]:
##positivereviews

In [19]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [20]:
count_positivetrain = {}
for bag in positivereviews:
    for word in bag:
        if word not in count_positivetrain:
            count_positivetrain[word]=1
        else: 
            count_positivetrain[word]+=1
##print(sorted(list(count_positivetrain.items()), key=lambda x: x[1], reverse=True)) 

In [21]:
numtrue_positive = 0
numtrue_negative = 0
numfalse_positive = 0
numfalse_negative = 0
for _,row in test.iterrows():
    review = row["review"]
    ##review = "it was the best. loved it."
    review = review.replace("<br />", " ")
    doc = nlp(review)
    tokenbyreview = []
    ##print(review)
    for token in doc:
        if token.is_stop==False and token.is_punct==False and token.text.isnumeric()==False:
            tokenbyreview.append(token.text.lower())
    tokenbyreview = set(tokenbyreview)
    score = 0
    for word in tokenbyreview: 
        if word in count_positivetrain: 
            score+=count_positivetrain[word]
        if word in count_negativetrain: 
            score-=count_negativetrain[word]
    ##print(score)
    if (score>0 and row["sentiment"] == "positive"): numtrue_positive+=1
    if (score<0 and row["sentiment"] == "negative"): numtrue_negative+=1
    if (score<0 and row["sentiment"] == "positive"): numfalse_negative+=1
    if (score>0 and row["sentiment"] == "negative"): numfalse_positive+=1    
print(f"Number of true positives: {numtrue_positive}, or {numtrue_positive/len(test)*100}%")
print(f"Number of true negatives: {numtrue_negative}, or {numtrue_negative/len(test)*100}%")
print(f"Number of false positives: {numfalse_positive}, or {numfalse_positive/len(test)*100}%")
print(f"Number of false negatives: {numfalse_negative}, or {numfalse_negative/len(test)*100}%")
accuracy = (numtrue_positive+numtrue_negative)/len(test)
print(accuracy)

Number of true positives: 3126, or 31.259999999999998%
Number of true negatives: 4560, or 45.6%
Number of false positives: 389, or 3.8899999999999997%
Number of false negatives: 1925, or 19.25%
0.7686
